# Trực quan hoá khoảng cách từ khách hàng đến cửa hàng

## Cài đặt thư viện cần thiết

In [1]:
import folium
import math
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from IPython.display import display, IFrame

import warnings
warnings.filterwarnings("ignore")

## Đọc dữ liệu

In [2]:
dtype = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str
}

df_customer = pd.read_csv("../../data/preprocessed/customer_eng.csv", dtype=dtype)
df_store = pd.read_csv("../../data/preprocessed/store_eng.csv", dtype=dtype)
df_geocode = pd.read_csv("../../data/preprocessed/geocode_eng.csv", dtype=dtype)

## Tính khoảng cách giữa khách hàng và cửa hàng

In [3]:
def calc_distance(lat1, lon1, lat2, lon2):
    """
    Tính khoảng cách Haversine giữa hai điểm trên Trái Đất dựa trên vĩ độ và kinh độ.
    
    Tham số:
    lat1 (float): Vĩ độ của điểm thứ nhất.
    lon1 (float): Kinh độ của điểm thứ nhất.
    lat2 (float): Vĩ độ của điểm thứ hai.
    lon2 (float): Kinh độ của điểm thứ hai.
    
    Trả về:
    float: Khoảng cách giữa hai điểm tính bằng km, làm tròn đến một chữ số thập phân.
    """
    R = 6371  # Bán kính Trái Đất tính bằng km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])  # Chuyển đổi độ sang radian
    
    distance = R * math.acos(
        math.sin(lat1) * math.sin(lat2) +
        math.cos(lat1) * math.cos(lat2) * math.cos(lon1 - lon2)
    )
    
    return np.round(distance, 1)

In [4]:
# Nhóm dữ liệu geocode theo mã bưu điện và tính giá trị trung bình của vĩ độ và kinh độ cho mỗi nhóm
df_customer_combined = (
    df_geocode.groupby("postal_cd").agg(
        m_longitude=("longitude", "mean"),
        m_latitude=("latitude", "mean")
    )
    .reset_index()
    # Kết hợp với dữ liệu khách hàng dựa trên mã bưu điện
    .merge(df_customer, how="inner", on="postal_cd")
    # Kết hợp với dữ liệu cửa hàng dựa trên mã cửa hàng
    .merge(df_store, how="left", left_on="application_store_cd", right_on="store_cd")
    # Đổi tên các cột địa chỉ cho dễ hiểu
    .rename(columns={
        "address_x": "customer_address",
        "address_y": "store_address"
    })
    # Tính khoảng cách giữa khách hàng và cửa hàng
    .assign(distance=lambda df: df.apply(
        lambda row: calc_distance(row['m_latitude'], row['m_longitude'], row['latitude'], row['longitude']),
        axis=1)
    )
    # Chọn các cột cần thiết cho DataFrame cuối cùng
    [[
        'customer_id',
        'customer_name',
        'customer_address',
        'm_longitude',
        'm_latitude',
        'store_name',
        'store_address',
        'prefecture',
        'longitude',
        'latitude',
        'distance'
    ]]
)

# Hiển thị 10 hàng đầu tiên của DataFrame kết quả
df_customer_combined.head(10)

,customer_id,customer_name,customer_address,m_longitude,m_latitude,store_name,store_address,prefecture,longitude,latitude,distance
0,CS005713000021,Ruria Noda,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Shirasagi store,"Shirasagi Sanchome, Nakano-ku, Tokyo",Tokyo,139.6307,35.72246,0.7
1,CS004513000232,Reiko Fujishima,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Sagimiya store,"Saginomiya Sanchome, Nakano-ku, Tokyo",Tokyo,139.6421,35.72348,1.7
2,CS005415000202,Nana Ishiguro,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Shirasagi store,"Shirasagi Sanchome, Nakano-ku, Tokyo",Tokyo,139.6307,35.72246,0.7
3,CS004215000052,Yumiko Senoo,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Sagimiya store,"Saginomiya Sanchome, Nakano-ku, Tokyo",Tokyo,139.6421,35.72348,1.7
4,CS005415000361,Miyuki Kawahara,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Shirasagi store,"Shirasagi Sanchome, Nakano-ku, Tokyo",Tokyo,139.6307,35.72246,0.7
5,CS005415000557,Natsumi Uno,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Shirasagi store,"Shirasagi Sanchome, Nakano-ku, Tokyo",Tokyo,139.6307,35.72246,0.7
6,CS004215000095,Yoko Arai,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Sagimiya store,"Saginomiya Sanchome, Nakano-ku, Tokyo",Tokyo,139.6421,35.72348,1.7
7,CS005415000563,Hikaru Ishii,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Shirasagi store,"Shirasagi Sanchome, Nakano-ku, Tokyo",Tokyo,139.6307,35.72246,0.7
8,CS005415000025,Rary,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Shirasagi store,"Shirasagi Sanchome, Nakano-ku, Tokyo",Tokyo,139.6307,35.72246,0.7
9,CS004412000362,Yu Tamura,"Suginami -ku, Tokyo Shimoigusa",139.62378,35.72037,Sagimiya store,"Saginomiya Sanchome, Nakano-ku, Tokyo",Tokyo,139.6421,35.72348,1.7


In [5]:
# Tạo bản đồ với tâm là vị trí trung bình của các điểm dữ liệu
center_lat = df_customer_combined[['m_latitude', 'latitude']].mean().mean()
center_lon = df_customer_combined[['m_longitude', 'longitude']].mean().mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Thêm các điểm dữ liệu vào bản đồ
for _, row in df_customer_combined.iterrows():
    # Điểm khách hàng
    folium.Marker(
        location=[row['m_latitude'], row['m_longitude']],
        popup=f"Customer ID: {row['customer_id']}<br>Address: {row['customer_address']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)
    
    # Điểm cửa hàng
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Store Name: {row['store_name']}\nStore Address: {row['store_address']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)
    
    # Đường nối giữa khách hàng và cửa hàng
    folium.PolyLine(
        locations=[(row['m_latitude'], row['m_longitude']), (row['latitude'], row['longitude'])],
        color='green'
    ).add_to(m)
    
    # Thêm khoảng cách
    folium.Marker(
        location=[(row['m_latitude'] + row['latitude']) / 2, (row['m_longitude'] + row['longitude']) / 2],
        popup=f"Distance: {row['distance']} km",
        icon=folium.DivIcon(html=f"""<div style="font-size: 12pt">{row['distance']} km</div>""")
    ).add_to(m)

# Lưu bản đồ vào file HTML
map_path = 'map.html'
m.save(map_path)

# Hiển thị bản đồ trong Jupyter Lab
display(IFrame(map_path, width=800, height=600))